# Deep Learning in Medicine Spring 2025

## Homework 3 - Sequence modeling

Hi! This homework aims to let you get acquianted with how sequence modelling in Deep Learning can be utilized for the biology domain. 

We will be utilizing existing public datasets and performing 4 separate steps with Torch.

Our data is related to enhancers, genomic elements which can alter the levels of gene expression.

There are 4 tasks:

1. RNNs/LSTMs for sequence modelling - You will be tasked to utilize existing PyTorch infrastructure and LSTMs to perform sequence classification (40 points).

2. Transformers - Implement a transformer with a classification head, compare and contrast with the LSTM architecture in terms of accuracy, memory requirements, and runtime. (40 points).

3. Hyperparameter optimization - Use one of your two Transformers alongside a hyperparameter optimization library to explore different hyperparameter and potentially increase performance. (30 points).

4. Extracting and exploring embeddings from pre-trained SSL models - Do the big models truly understand what is an enhancer? If so, their latent representations of such sequences should be meaningful. (30 points).

The maximum you can get for the whole homework is 140. This means that you can choose to solve two assignments fully and one partially depending on your interests and/or familiarity.




### Dataset loading

Make sure you install the [https://github.com/ML-Bioinfo-CEITEC/genomic_benchmarks](genomic-benchmarks) package with ```pip install genomic-benchmarks```. Make sure your gdown package is updated!

#### Explore the dataset

In [ ]:
from genomic_benchmarks.data_check import list_datasets
from genomic_benchmarks.data_check import info
from genomic_benchmarks.loc2seq import download_dataset

info('human_enhancers_ensembl')



: 

As you can see the dataset is quite large, so make sure you have GPU access when using it.

#### Download the dataset

In [ ]:
 download_dataset("human_enhancers_ensembl") #51MB zipped, 600 MB full

#### Load the dataset

Luckily the repository gives us ample instructions on how to load the dataset

In [ ]:
from genomic_benchmarks.dataset_getters.pytorch_datasets import HumanEnhancersEnsembl
train = HumanEnhancersEnsembl(split='train', version=0)
test = HumanEnhancersEnsembl(split='test', version=0)

#### Dataset exploration (5 points)

Time to take a look at the data!

Use your code to check:

1. Number of examples in train and test datasets (A simple print will do)
2. Structure of examples (You can use)
3. Maximum of sequences (Important padding length!)
4. Number of nucleotides, di-nucleotides, trinucleotides (Tip: You can use defaultdict!).

In [ ]:
print("Number of train examples:", len(train))
print("Number of test examples:", len(test))

print("Structure of a training example:", train[0])

import collections
max_len = 0
nuc_counts = collections.defaultdict(int)
dinuc_counts = collections.defaultdict(int)
trinuc_counts = collections.defaultdict(int)

for sample in train:
    seq = sample[0] 
    max_len = max(max_len, len(seq))
    for char in seq:
        nuc_counts[char] += 1
    for i in range(len(seq) - 1):
        dinuc_counts[seq[i:i+2]] += 1
    for i in range(len(seq) - 2):
        trinuc_counts[seq[i:i+3]] += 1

print("Maximum sequence length (train):", max_len)
print("Nucleotide counts:", dict(nuc_counts))
print("Di-nucleotide counts:", dict(dinuc_counts))
print("Tri-nucleotide counts:", dict(trinuc_counts))

#### Generate a separate dataset with non-overlapping n-nucleotides (5 points)

We will use those to test the difference of utilizing different levels of encoding our bases (by 1, 2, 3).

Make sure to generate the vocabulary and any special tokens you might need.

In [ ]:
# Your code goes here:
def generate_n_nucleotides_dataset(train, n):
    """
    Generate a dataset with non-overlapping n-nucleotides and apply padding.
    Args:
        train: Original training dataset.
        n: Length of n-nucleotides.
    Returns:
        transformed_dataset: List of transformed sequences with n-nucleotides.
        vocab: Vocabulary of n-nucleotides.
    """
    import collections

    transformed_dataset = []
    vocab = set()
    special_tokens = {"<PAD>", "<UNK>"}
    vocab.update(special_tokens)

    max_seq_len = 0  # Track the maximum sequence length for padding

    for sample in train:
        seq = sample[0]
        n_nucleotides = [seq[i:i+n] for i in range(0, len(seq), n) if len(seq[i:i+n]) == n]
        max_seq_len = max(max_seq_len, len(n_nucleotides))  # Update max sequence length
        transformed_dataset.append((n_nucleotides, sample[1]))
        vocab.update(n_nucleotides)

    # Apply padding to sequences
    for i in range(len(transformed_dataset)):
        n_nucleotides, label = transformed_dataset[i]
        padded_sequence = n_nucleotides + ["<PAD>"] * (max_seq_len - len(n_nucleotides))
        transformed_dataset[i] = (padded_sequence, label)

    vocab = sorted(vocab)  # Sort for consistency
    return transformed_dataset, vocab

import torch
from torch.utils.data import Dataset

class NucleotideDataset(Dataset):
    """
    PyTorch Dataset for n-nucleotide sequences.
    """
    def __init__(self, transformed_dataset, vocab):
        self.data = transformed_dataset
        self.vocab = {token: idx for idx, token in enumerate(vocab)}
        self.pad_token = "<PAD>"
        self.unk_token = "<UNK>"

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sequence, label = self.data[idx]
        encoded_sequence = [self.vocab.get(token, self.vocab[self.unk_token]) for token in sequence]
        return torch.tensor(encoded_sequence, dtype=torch.long), torch.tensor(label, dtype=torch.long)


n = 3  # Change this to 1, 2, or 3 as needed
transformed_train, vocab = generate_n_nucleotides_dataset(train, n)
transformed_test, vocab = generate_n_nucleotides_dataset(train, n)
train_dataset = NucleotideDataset(transformed_train, vocab)
test_dataset = NucleotideDataset(transformed_test, vocab)

# Example DataLoader
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, drop_last=False)
# Iterate through the DataLoader
for batch in train_loader:
    sequences, labels = batch
    print("Batch sequences:", sequences)
    print("Batch labels:", labels)
    break

### Train a PyTorch LSTM single-layer LSTM model to classify sequences (15 points)

In [ ]:
from torch.nn.functional import cross_entropy

import torch.nn as nn
import torch.optim as optim

# Define the LSTMClassifier class
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, vocab_size, pad_idx):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, input_size, padding_idx=pad_idx)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        _, (hidden, _) = self.lstm(x)
        out = self.fc(hidden[-1])
        return out



# Define training and testing loops
def train_model(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for sequences, labels in train_loader:
        sequences, labels = sequences.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    accuracy = correct / total
    return total_loss / len(train_loader), accuracy

# Define testing loop
def test_model(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for sequences, labels in test_loader:
            sequences, labels = sequences.to(device), labels.to(device)

            outputs = model(sequences)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    return total_loss / len(test_loader), accuracy

# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model parameters
input_size = 128  # Embedding size
hidden_size = 256
num_layers = 1
num_classes = len(set([label for _, label in transformed_train]))  # Number of unique labels
vocab_size = len(vocab)
pad_idx = vocab.index("<PAD>")

# Initialize model, optimizer, and loss function
model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes, vocab_size, pad_idx).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = cross_entropy

# Training and testing the model
num_epochs = 5
for epoch in range(num_epochs):
    train_loss, train_accuracy = train_model(model, train_bloader, optimizer, criterion, device)
    test_loss, test_accuracy = test_model(model, test_loader, criterion, device)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")



#### Calculate AUC on the testing set for your model and plot the loss curve (5 points)

In [ ]:

# Your code goes here:

#### Repeat this exercise using an n-nucleotide representation of your choosing. Compare and contrast the accuracy, AUC, stability, training time, memory requirements, and inference time of your model (10 points)

In [ ]:
# Your code goes here:

### Training a Transformer (40 points)

As you all know, the Transformer architecture has taken the world of Natural Language Processing by storm and the world of biology followed soon after. The transformer is the backbone upon which most of the necessary architectures are built nowadays.

In this part of the assignment, you are asked to build an initial transformer and iterate on it, through linearization of attention to reduce requirements and runtime, incredible engineering feats such as flash-attention as well as read through and implement some of the more academic versions through outside resources such as x-transformers and huggingface's libraries.

#### Train a vanilla transformer on a subset of the data (10 points)

You can use 1/10th, or 1/5th of the data depending on runtimes.

Evaluate the results on the test set.

Never forget your positional encodings!

In [ ]:
# Your code goes here:

#### Train a linearized transformer (your choice) on a subset of the data (10 points)

Use a linearized transformer and re-train. Juxtapose differences in memory and runtime requirements requirements, accuracy, AUC.

In [ ]:
# Your code goes here:

#### Train a transformer that uses Flash Attention (10 points)

In [ ]:
# Your code goes here:

#### Train a model that uses recent architectural improvements in transformers (10 points)

A great reference is here: https://github.com/lucidrains/x-transformers. You're free to use any combination of architecture and optimizer you would like to test. E.g. layers of BatchNorm, multiple transformer layers, MLA implementations, second-order optimizers, torch JIT compilation, etc. 

Please write a small paragraph about why you chose that specific architectural development. You are not required to use xtransformers, but please point me to the library you utilized and steps to install it.

Once again, compare and constrast for runtime, memory, accuracy, AUC.


In [ ]:
# Your code goes here:

### Hyperparameter optimization (30 points)

Sometimes you should be willing to give up some accuracy for portability (requirements - ease of running on consumer-grade hardware or "the edge") and usability (inference time - e.g. in case this would be running as a service).

In this part of the assignments, use a hyperparameter optimization library (preferably optuna) and the subset of the training data to find the best hyperparameters for either the LSTM or the transformer you implemented in the the previous parts of the assignment.

In this part of the assignment, you are aiming to find the best trade-off of accuracy, runtime, and requirements.

You are expected to manipulate the following hyperparameters (not exhaustively):

- Learning rate
- Number of layers/heads
- Hidden states
- Embedding size
- Dropout rate
- Batch normalization

After you have found the best hyperparameters, train the model on the full training dataset and report accuracy, AUC and running time, both for training.

Submissions will be judged based on the following criteria:

- Correctness of code (20 points)
- Training/Inference time and requirements (5 points)
- Total accuracy (5 points)

The latter two will be judged across the whole class.

All code will be tested on Tesla V100s (16GB) - Please do not use larger GPUs for your training



### Utilizing SSL models (30 points)

Many times in daily research, you are not required to train these huge models on massive corpa from scratch. Institutions, as well as companies, have been generating models, and data and training them. You can leverage these models to extract useful embeddings for downstream tasks.

In this part of the assignment, you are asked to utilize Evo, a foundation DNA language model to extract embeddings of your positive and negative sequences and then embark on a journey of data exploration.

The original Evo model can be found here: https://github.com/evo-design/evo

#### Import the evo package and successfully run the following example code (5 points):

In [ ]:
from evo import Evo
import torch

device = 'cuda:0'

evo_model = Evo('evo-1-131k-base')
model, tokenizer = evo_model.model, evo_model.tokenizer
model.to(device)
model.eval()

sequence = 'ACGT'
input_ids = torch.tensor(
    tokenizer.tokenize(sequence),
    dtype=torch.int,
).to(device).unsqueeze(0)

with torch.no_grad():
    logits, _ = model(input_ids) # (batch, length, vocab)

print('Logits: ', logits)
print('Shape (batch, length, vocab): ', logits.shape)

#### Generate embeddings from the model utilizing the subset of your sequences (5 points):

In [ ]:
# Your code goes here:

#### Exploratory analysis (15 points):




Compute a dimensionality reduction of your embeddings, colored by their label. You can use PCA/tSNE/UMAP. Visualize it and comment on cluster segregation. Additionally, for each of your sequences, color each sequence by their dominant n-nucleotide, if that n-nucleotide's counts accounts for above 0.5/n of all dinucleotides. (e.g. if you're doing dinucleotides, you use 0.25, trinucleotides you use 0.5/0.3, tetranucleotides you use 0.125, pentanucleotides 0.1 etc.). Comment on any interesting findings.

In [ ]:
# Your code goes here:

Utilize a clustering algorithm (e.g. K-means, Leiden, H-DBSCAN) to cluster the embeddings. Is the clustering reflected on the UMAP? Plot the proportions of your chosen n-nucleotide in each of the cluster. 

In [ ]:
# Your code goes here:

#### Train on top of the embeddings (10 points):

Did we even need the pre-trained transformer at all?

Training a multi-layer MLP on top of the embeddings to classify enhancer from non-enhancer sequences. How does it compare to the models you already trained based on accuracy, AUC, runtime and memory requirements?

In [ ]:
# You code goes here: